In [2]:
import os
import SQLAlchemy
SQLALCHEMY_SILENCE_UBER_WARNING = 1
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
def load_csv_from_drive(drive_url):
    url='https://drive.google.com/uc?id=' + drive_url.split('/')[-2]
    transactions = pd.read_csv(url)
    return transactions

drive_url = 'https://drive.google.com/file/d/1GraoErFP7JjE00wV4ERZrDg2Dtn8aASt/view?usp=sharing'
transactions = load_csv_from_drive(drive_url)

In [3]:
def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of MySQL. """
    db_host = "34.132.33.93" # e.g. '34.132.33.93' ('172.17.0.1' if deployed to GAE Flex)
    db_user = "root" # e.g. 'my-db-user'
    db_pass = "Pandas2020!" # e.g. 'my-db-password'
    db_name = "database" # e.g. 'my-database'
    db_port = 3306 # e.g. 3306

    engine = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
    )
    return engine

engine = connect_tcp_socket() 
conn = engine.connect()

In [ ]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)

In [10]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [7]:
transactions.head()

,"<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><style nonce=""erfZ-tZavLDiS7omknoDwA"">.goog-inline-block{position:relative;display:-moz-inline-box;display:inline-block}* html .goog-inline-block{display:inline}*:first-child+html .goog-inline-block{display:inline}.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial",sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,".uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;text-align:center}.uc-footer a{color:#15c}.uc-footer a:visited{color:#61c}.uc-footer a:active{color:#d14836}.uc-footer-divider{color:#ccc;width:100%}sentinel{}</style><link rel=""icon"" href=""//ssl.gstatic.com/docs/doclist/images/drive_2022q3_32dp.png""/></head><body><div class=""uc-main""><div id=""uc-dl-icon"" class=""image-container""><div class=""drive-sprite-aux-download-file""></div></div><div id=""uc-text""><p class=""uc-warning-caption"">Google Drive can't scan this file for viruses.</p><p class=""uc-warning-subcaption""><span class=""uc-name-size""><a href=""/open?id=1GraoErFP7JjE00wV4ERZrDg2Dtn8aASt"">transactions.csv</a> (3.2G)</span> is too large for Google to scan for viruses. Would you still like to download this file?</p><form id=""download-form"" action=""https://drive.google.com/uc?id=1GraoErFP7JjE00wV4ERZrDg2Dtn8aASt&amp;confirm=t&amp;uuid=5b7078f9-d2ab-4f98-aa89-032c72f79584"" method=""post""><input type=""submit"" id=""uc-download-link"" class=""goog-inline-block jfk-button jfk-button-action"" value=""Download anyway""/></form></div></div><div class=""uc-footer""><hr class=""uc-footer-divider""></div></body></html>"


In [6]:
transactions.select_dtypes(include=['int64']).columns
transactions.select_dtypes(include=['float64']).columns
transactions.select_dtypes(include=['object']).columns

Index(['prod_name', 'product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_code', 'index_name', 'index_group_name',
       'section_name', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [ ]:
# Printing the amount of features available in total
count_numerical = len(transactions.select_dtypes(include=['int64','float64']).columns)
count_categorical = len(transactions.select_dtypes(include=['object']).columns)
count_total = count_categorical + count_numerical
print('Total Features: ', count_categorical, 'categorical', '+',
      count_numerical, 'numerical', '=', count_total, 'features')

In [ ]:
# Detecting the number of missing or null values per variable
pd.concat([transactions.isnull().sum(),transactions.isna().sum(min_count=1)],keys=['Nulls','Empty'],axis=1).sort_values(by='Nulls', ascending=False)[:20]

In [ ]:
# Changing dates to datetimes
transactions["t_dat"] = \
    pd.to_datetime(transactions["t_dat"])

In [ ]:
# Changing column naem
transactions.rename(columns = {'t_dat':'date'}, inplace = True)

In [ ]:
# Uploading data to database
transactions.to_sql(name = "transactions", con = conn, if_exists = 'fail', index = False) # no index will be uploaded to database as False

In [ ]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r) 